In [1]:
from bs4 import BeautifulSoup
import bs4
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import glob
import os
import sys
from dotenv import load_dotenv


sys.path.insert(1, os.getenv("PROJECT_PATH") + '/utils/')

import spacy_util


def delete_csv_files():
    files = glob.glob('*.csv')
    for f in files:
        os.remove(f)


def init_livecareer(driver: webdriver, job_list: list, pages: int=5) -> pd.DataFrame:

    # CREATING A DATAFRAME TO STORE THE LINKS OF EACH INDIVIDUAL RESUME EXAMPLE
    resume_links = pd.DataFrame()
    profile = []
    url = []

    # LOOP TO SEARCH FOR RESUME EXAMPLES FROM THE WEBPAGE AS PER THE LABELS DEFINED ABOVE. ONLY (10*12) 120 RESUMES WILL BE COLLECTED.

    # REMOVE  "&bg=85&eg=100&comp=&mod="  TO EXPAND THE SEARCH 
    # (bg=85 & eg=100) FILTERS THE RESUME BASED ON SCORE OF BETWEEEN 85 AND 100, INCREASE RANGE TO EXPAND SEARCH

    for job in job_list:  
        
        
        for i in range(1,pages + 1):   # INCREASE THE RANGE TO GET MORE RESUME DATA - 13 PAGES OF 10 RESUMES EACH
            PAGE = str(i)
            URL = "https://www.livecareer.com/resume-search/search?jt=" + job + "&bg=85&eg=100&comp=&mod=&pg=" + PAGE
            driver.delete_all_cookies()
            driver.get(URL)
            aTagsInLi = driver.find_elements(By.CSS_SELECTOR,"div a")
            
            for a in aTagsInLi:            
                if a.get_attribute('rel') == "noopener noreferrer" and "https://www.livecareer.com/resume-search/r/" in a.get_attribute('href'):
                    profile.append(job)
                    url.append(a.get_attribute('href'))
        
    # STORES THE CATEGORY AND LINK TO THE RESUME WEBPAGE

    resume_links["profile"] = profile
    resume_links["url"] = url

    # HASHES THE LINK AND CREATES AN UNIQUE ID FOR THE ROWS

    import hashlib
    def id(x):
        return int(hashlib.md5(x.encode('utf-8')).hexdigest(), 16)

    resume_links["id"] = resume_links["url"].apply(id)
    resume_links["data"] = ""
    resume_links["resume_html"] = ""    
    
    return resume_links


def get_details(resume_links: pd.DataFrame, driver: webdriver) -> pd.DataFrame:
    driver.delete_all_cookies()

    for i in range(resume_links.shape[0]):
        url = resume_links.url[i]
        print(url)
    
        driver.get(url)
        #time.sleep(0.5)                  #ADDED DELAY, CAN BE REMOVED
        x = driver.page_source     
        x = x.replace(">","> ")
        soup = bs4.BeautifulSoup(x, 'html.parser')  
        
        div = soup.find("div", {"id": "server-resume-template"})   
            
        resume_links.resume_html[i] = "none"
        try:
            resume_links.data[i] = div.getText().replace("|"," ").replace("\n"," ").strip()
            
        except:
    #         ADD EXCEPTION IF REQUIRED
            pass
    
    return resume_links

In [6]:
# LIST OF UNIQUE CATEGORIES THAT WAS NEEDED FOR THE DATASET.
# ADD OR SUBTRACT LABELS AS NEEDED
#['Data Analyst','Data Engineer','Data Scientist']

#Out of scope - We need more data for less profiles
#['Business Analyst','Software Engineer','Machine Learning Engineer','Cloud Engineer']

output_file = "livecareer-data.csv"
job_list =['Data Scientist']
pages = 7


# USING SELENIUM FOR WEB SCRAPPING
driver = webdriver.Chrome()

df = init_livecareer(driver, job_list,pages)

df = get_details(df, driver)

delete_csv_files()

df['data'] = df['data'].apply(str)
df['data'] = df['data'].apply(spacy_util.remove_punctuation)  

# STORES THE DATAFRAME
df.to_csv(output_file,sep='|',index=False, encoding='utf-8')

# CLOSES THE BROWSER
driver.quit()

#NER --- df['data']

https://www.livecareer.com/resume-search/r/data-scientist-2a3791654fd140e2ad9f230d470f1043
https://www.livecareer.com/resume-search/r/data-scientist-001599f915214ef19135cfa5398e06b8
https://www.livecareer.com/resume-search/r/data-scientist-b4624bc712c04044834adfff057941ab
https://www.livecareer.com/resume-search/r/data-scientist-0701abab38a940ccbd523ad6b6ba8810
https://www.livecareer.com/resume-search/r/graduate-assistant-data-scientist-30e747ef925940eb909e346892bc157e
https://www.livecareer.com/resume-search/r/data-scientist-consultant-26d8093a10f14c83bb8218b443e4f969
https://www.livecareer.com/resume-search/r/senior-data-scientist-298f59e78c104fc0bcfa7b8e3d3f43d6
https://www.livecareer.com/resume-search/r/data-scientist-tech-director-dba-of-citeseerx-a1a1abff4a8e4074ad01b0cc6b661bd7
https://www.livecareer.com/resume-search/r/research-assistant-data-scientist-a0ac60734008486993ec47865153be1d
https://www.livecareer.com/resume-search/r/data-scientist-project-manager-ceo-73538fa67f974e75